# Progetto Lar Generators

| Nome| Matricola | E-mail | Profilo Github |
|:---:|:---:|:---:|:---:|
|Paolo Mazzitti|502042|pao.mazzitti@stud.uniroma3.it| [https://github.com/paolomazzitti](https://github.com/paolomazzitti) |
| Matteo Colonnello|527289|mat.colonnello@stud.uniroma3.it|[https://github.com/MatteoColonnello](https://github.com/MatteoColonnello)|
| Martina Falanga|522705|mar.falanga@stud.uniroma3.it|[https://github.com/MartinaFalanga](https://github.com/MartinaFalanga) |

Per avviare una istanza Jupyter con 8 threads, eseguire il seguente codice dal proprio terminale e selezionare il kernel appena creato.

```julia
using IJulia
IJulia.installkernel("Julia 8 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "8",
))
````


In [27]:
using Base.Threads
Threads.nthreads()

8

## Codice funzionante versione originale
Per mostrare le differenze prestazionali tra la versione originale del progetto e quella ottimizzata dal gruppo è necessario eseguire la cella sottostante. Il codice, infatti, presentava inizialmente degli errori che non ne permettavano l'esecuzione.

In [28]:
function chainbasis2solidsEx(V,copEV,copFE,copCF)
    CF = [findnz(copCF[k,:])[1] for k=1:copCF.m]
    FE = [findnz(copFE[k,:])[1] for k=1:copFE.m]
    EV = [findnz(copEV[k,:])[1] for k=1:copEV.m]

    FEs = []
    EVs = []
    FVs = []
    for k=1:copCF.m
        push!( FEs, [collect(Set(GL.Cat([e for e in FE[f]]))) for f in CF[k]] )
        push!( EVs, [[EV[e] for e in FE[f]] for f in CF[k]] )
        push!( FVs, [collect(Set(GL.Cat([EV[e] for e in FE[f]]))) for f in CF[k]] )
    end
    pols = collect(zip(EVs,FVs,FEs))
    W = convert(Lar.Points,V')
    return W,pols,CF
end

function internalpointsEx(V,copEV,copFE,copCF)

    U,pols,CF = chainbasis2solidsEx(V,copEV,copFE,copCF)

    innerpoints = []
    intersectedfaces = []
    for k=1:length(pols)
        (EV,FV,FE),Fs = pols[k],CF[k]
        EV = convert(Lar.Cells,collect(Set(GL.Cat(EV))))
        points,facenumber = Lar.getinternalpoint(V,EV,FV,Fs, copEV,copFE)
        push!(innerpoints,points)
        push!(intersectedfaces,facenumber)
    end
    return innerpoints,intersectedfaces
end

function bool3dEx(assembly)

    V,FV,EV = Lar.struct2lar(assembly)
    cop_EV = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells));
    cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells);
    W = convert(Lar.Points, V');
    V, copEV, copFE, copCF = Lar.space_arrangement( W, cop_EV, cop_FE)
    W = convert(Lar.Points, V');

    innerpoints,intersectedfaces = internalpointsEx(W,copEV,copFE,copCF[2:end,:])

    listOfModels = Lar.evalStruct(assembly)
    inputfacenumbers = [length(listOfModels[k][2]) for k=1:length(listOfModels)]
    cumulative = cumsum([0;inputfacenumbers]).+1
    fspans = collect(zip(cumulative[1:end-1], cumulative[2:end].-1))
    span(h) = [j for j=1:length(fspans) if fspans[j][1]<=h<=fspans[j][2] ]

    V,FV,EV = Lar.struct2lar(assembly)
    containmenttest = Lar.testinternalpoint(V,EV,FV)

    boolmatrix = BitArray(undef, length(innerpoints)+1, length(fspans)+1)
    boolmatrix[1,1] = 1
    for (k,point) in enumerate(innerpoints)
        cells = containmenttest(point)

        rows = [span(h) for h in cells]
        for l in GL.Cat(rows)
            boolmatrix[k+1,l+1] = 1
        end
    end
    return W, (copEV, copFE, copCF), boolmatrix
end

bool3dEx (generic function with 1 method)

## Codice sviluppato e ottimizzato

In [29]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation; 
using IntervalTrees, LinearAlgebra, SparseArrays
using Base.Threads

In [30]:
"""
    setTileBool(box)(point)

# Explanation

Returns the tile code of an edge.

# Arguments

- `box::Tuple{Float64, Float64, Float64, Float64}`
- `point::Vector{Float64}`

# Return

- `tileCodeBool::Int64`
"""
@inline function setTileBool(box)
    b1, b2, b3, b4 = box
    function tileCodeBool(point)
        x, y = point
        code = 0
        if y > b1
            code = code | 1
        end
        if y < b2
            code = code | 2
        end
        if x > b3
            code = code | 4
        end
        if x < b4
            code = code | 8
        end
        return code
    end
    return tileCodeBool
end

setTileBool

In [31]:
"""
    pointInPolygonClassificationBool(V, EV)(pnt)

# Explanation

Determines whether a point is inside or outside a polygon. It is a specific version used for the bool2d function.

# Arguments
- `V::Adjoint{Float64, Matrix{Float64}}`
- `EV::Vector{Vector{Int64}}`
- `pnt::Vector{Float64}`

# Return
- `p_in::String` or `p_out::String`
"""
function pointInPolygonClassificationBool(V, EV)
    function pointInPolygonClassification0Bool(pnt)
        x, y = pnt
        xmin, xmax, ymin, ymax = x, x, y, y
        tilecodeBool = setTileBool([ymax, ymin, xmax, xmin])
        count = 0
        for edge in EV
            p1, p2 = V[:, edge[1]], V[:, edge[2]]
            c1, c2 = tilecodeBool(p1), tilecodeBool(p2)
            (x1, y1), (x2, y2) = p1, p2
            c_edge, c_int = c1 ⊻ c2, c1 & c2

            if c_edge == 3 && c_int == 4
                count += 1
            elseif c_edge == 15
                x_int = ((y - y2) * (x1 - x2) / (y1 - y2)) + x2
                if x_int > x
                    count += 1
                end
            end
        end
        if (round(count) % 2) == 1
            return "p_in"
        else
            return "p_out"
        end
    end
    return pointInPolygonClassification0Bool
end

pointInPolygonClassificationBool

In [32]:
"""
	spaceindex(point3d)(model)

Compute the set of face boxes of possible intersection with a point-ray.
Work in 3D, where the ray direction is parallel to the z-axis.
Return an array of indices of face.

# Usage
```jldoctest
julia> V,(VV,EV,FV,CV) = Lar.cuboidGrid([1,1,1],true)

julia> spaceindex([.5,.5,.5])((V,FV))
3-element Array{Int64,1}:
 5
 6
```
"""
function spaceindex(point3d::Array{Float64,1})::Function
    function spaceindex0(model::Lar.LAR)::Array{Int,1}
        V, CV = copy(model[1]), copy(model[2])
        V = [V point3d]
        dim, idx = size(V)
        push!(CV, [idx, idx, idx])
        cellpoints = [V[:, CV[k]]::Lar.Points for k = 1:length(CV)]
        #----------------------------------------------------------
        bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
        xboxdict = Lar.coordintervals(1, bboxes)
        yboxdict = Lar.coordintervals(2, bboxes)
        # xs,ys are IntervalTree type
        xs = IntervalTrees.IntervalMap{Float64,Array}()
        for (key, boxset) in xboxdict
            xs[tuple(key...)] = boxset
        end
        ys = IntervalTrees.IntervalMap{Float64,Array}()
        for (key, boxset) in yboxdict
            ys[tuple(key...)] = boxset
        end
        xcovers = Lar.boxcovering(bboxes, 1, xs)
        ycovers = Lar.boxcovering(bboxes, 2, ys)
        covers = [intersect(pair...) for pair in zip(xcovers, ycovers)]

        # remove each cell from its cover
        pointcover = setdiff(covers[end], [idx + 1])
        return pointcover[1:end-1]
    end
    return spaceindex0
end

spaceindex

In [33]:
"""
	rayintersection(point3d::Array{Float64})(V,FV,face::Int)

# Explanation
Compute the intersection point of the vertical line through `point3d` w `face`.
If the face is parallel to `z axis` return `false`.
# Usage
```jldoctest
julia> V,(VV,EV,FV,CV) = Lar.simplex(3,true);

julia> V
3×4 Array{Float64,2}:
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

julia> FV
4-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [1, 2, 4]
 [1, 3, 4]
 [2, 3, 4]

 julia> Lar.rayintersection([.333,.333,0])(V,FV,4)
 3-element Array{Float64,1}:
  0.333
  0.333
  0.3340000000000001
```
"""
function rayintersection(point3d)
    function rayintersection0(V, FV, face::Int)
        l0, l = point3d, [0, 0, 1.0]
        ps = V[:, FV[face]]
        p0 = ps[:, 1]
        v1, v2 = ps[:, 2] - p0, ps[:, 3] - p0
        n = LinearAlgebra.normalize(cross(v1, v2))

        denom = dot(n, l)
        if (abs(denom) > 1e-8)
            p0l0 = p0 - l0
            t = dot(p0l0, n) / denom
            if t > 0
                return l0 + t * l
            end
        else
            return false
        end
    end
    return rayintersection0
end

rayintersection

In [34]:
"""
	planemap(V,copEV,copFE,face)(point)

# Arguments

- `V::Matrix{Float64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`
- `face::Int64`

# Explanation

Tranform the 3D face and the 3D point in their homologous 2D, in order to test for containment.
"""
function planemap(V, copEV, copFE, face)
    fv, edges = Lar.vcycle(copEV, copFE, face)
    function planemap0(point)
        vs = V[:, fv]
        point = point .- vs[:, 1]
        vs = vs .- vs[:, 1]
        u, v = edges[1]
        z, w = [[z, w] for (z, w) in edges if z == v][1]
        v1 = vs[:, u] - vs[:, v]
        v2 = vs[:, w] - vs[:, v]
        v3 = cross(v2, v1)
        M = [v1 v2 v3]
        vs = inv(M) * [vs point]
        outvs = vs[1:2, 1:end-1]
        outpoint = vs[1:2, end]
        return outvs, edges, outpoint
    end
    return planemap0
end

planemap

In [35]:
"""
    settestpoints(V, FV, Fs, copEV, copFE)
    
# Arguments
- `V::Matrix{Float64}`
- `FV::Vector{Vector{Int64}}`
- `Fs::Vector{Int64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`

# Return
- `(ptest1, ptest2)::Tuple{Vector{Float64}, Vector{Float64}}`
"""
function settestpoints(V, FV, Fs, copEV, copFE)
    fdict = Dict(zip(Fs, 1:length(Fs)))
    countKey = 1
    f = Fs[1]
    e = findnz(copFE[f, :])[1][1] 
    f1 = findnz(copFE[:, e])[1][countKey]
    while !haskey(fdict, f1)
        countKey = countKey + 1
        f1 = findnz(copFE[:, e])[1][countKey]
    end
    f2 = findnz(copFE[:, e])[1][countKey+1]
    while !haskey(fdict, f2)
        countKey = countKey + 1
        f2 = findnz(copFE[:, e])[1][countKey]
    end
    v1, v2 = findnz(copEV[e, :])[1] 
    V1 = FV[fdict[f1]]
    V2 = FV[fdict[f2]]
    v1, v2 = intersect(V1, V2)
    t1 = V[:, v1], V[:, v2], V[:, [v for v in V1 if v ≠ v1 && v ≠ v2][1]]
    t2 = V[:, v2], V[:, v1], V[:, [v for v in V2 if v ≠ v1 && v ≠ v2][1]]
    n1 = LinearAlgebra.normalize(cross(t1[2] - t1[1], t1[3] - t1[1]))
    n2 = LinearAlgebra.normalize(cross(t2[2] - t2[1], t2[3] - t2[1]))
    p0 = (V[:, v1] + V[:, v2]) ./ 2
    n = n1 + n2
    ϵ = 1.0e-4
    ptest1 = p0 + ϵ * n
    ptest2 = p0 - ϵ * n
    return ptest1, ptest2
end

settestpoints

In [36]:
"""
	testinternalpoint(V, EV, FV)

# Arguments
- `V::Matrix{Float64}`
- `EV::Vector{Vector{Int64}}`
- `FV::Vector{Vector{Int64}}`

# Return
- `intersectedfaces::Vector{Int64}`
"""
function testinternalpoint(V, EV, FV)
    copEV = Lar.lar2cop(EV)
    copFV = Lar.lar2cop(FV)
    copFE = copFV * copEV'
    I, J, Val = findnz(copFE)
    triple = zip([(i, j, 1) for (i, j, v) in zip(I, J, Val) if v == 2]...)
    I, J, Val = map(collect, triple)
    Val = convert(Array{Int8,1}, Val)
    copFE = sparse(I, J, Val)
    function testinternalpoint0(testpoint)

        intersectedfaces = []

        faces = spaceindex(testpoint)((V, FV))
        depot = []

        for face in faces
            value = rayintersection(testpoint)(V, FV, face)
            if typeof(value) == Array{Float64,1}
                push!(depot, (face, value))
            end
        end

        for (face, point3d) in depot
            vs, edges, point2d = planemap(V, copEV, copFE, face)(point3d)
            classify = pointInPolygonClassificationBool(vs, edges)
            inOut = classify(point2d)
            if inOut != "p_out"
                push!(intersectedfaces, face)
            end
        end
        return intersectedfaces
    end
    return testinternalpoint0
end

testinternalpoint

In [37]:
"""
    getinternalpoint(V,EV,FV,Fs, copEV,copFE,z,ptest1,ptest2)

# Arguments
- `V::Matrix{Float64}`
- `EV::Vector{Vector{Int64}}`
- `FV::Vector{Vector{Int64}}`
- `Fs::Vector{Int64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`
- `z::Vector{Any}`
- `ptest1::Vector{Int64}`
- `ptest2::Vector{Int64}`

# Return
- `ptest1::Vector{Int64}` or `ptest2::Vector{Int64}`

# Explanation
- For each test point compute the face planes intersected by vertical ray
- Transform each plane in 2D and look whether the intersection point is internal
- Return the test point with odd numeber of ray intersections
"""
function getinternalpoint(V, FV, Fs, copEV, copFE, z, ptest1, ptest2)

    for (f, face) in collect(enumerate(Fs))
        ret1 = rayintersection(ptest1)(V, FV, f)
        if typeof(ret1) == Array{Float64,1}
            push!(z[threadid()], (face, ret1))
        end
    end

    k1 = 0
    for (face, point3d) in z[threadid()]
        vs, edges, point2d = planemap(V, copEV, copFE, face)(point3d)
        classify = pointInPolygonClassificationBool(vs, edges)
        inOut = classify(point2d)
        if inOut == "p_in"
            k1 += 1
        end
    end
    if k1 % 2 == 1
        return ptest1
    else
        return ptest2
    end
end

getinternalpoint

In [38]:
"""
    chainbasis2solids(copEV, copFE, copCF)

# Arguments
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`
- `copCF::SparseMatrixCSC{Int8, Int64}`

# Return
- (`FVs, CF)::Tuple{Vector{Vector{Vector{Int64}}}, Vector{Vector{Int64}}}`
"""
function chainbasis2solids(copEV, copFE, copCF)
    CF = [findnz(copCF[k, :])[1] for k = 1:copCF.m]
    FE = [findnz(copFE[k, :])[1] for k = 1:copFE.m]
    EV = [findnz(copEV[k, :])[1] for k = 1:copEV.m]

    FVs = Vector{Vector{Vector{Int64}}}(undef, copCF.m)

    @threads for k = 1:copCF.m 
        FVs[k] = [collect(Set(GL.Cat([EV[e] for e in FE[f]]))) for f in CF[k]]
    end

    return FVs, CF
end

chainbasis2solids

In [39]:
"""
    internalpoints(V, copEV, copFE, copCF)
# Arguments
- `V::Matrix{Float64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`
- `copCF::SparseMatrixCSC{Int8, Int64}`

# Return
- `innerpoints::Vector{Any}`

# Explanation
- Transform each 3-cell in a solid
- Compute, for each 3-cell, one *internal point*
"""
function internalpoints(V, copEV, copFE, copCF)

    FVs, CF = chainbasis2solids(copEV, copFE, copCF)

    innerpoints = Array{Any}(undef, length(FVs))
    ptestArray = Array{Any}(undef, length(FVs))

    z = Vector{Any}()

    for _ in 1:Threads.nthreads()
        push!(z, Any[])
    end

    for l in 1:length(FVs)
        ptestArray[l] = settestpoints(V, FVs[l], CF[l], copEV, copFE)
    end

    for k in 1:length(FVs) 
        points = getinternalpoint(V, FVs[k], CF[k], copEV, copFE, z, ptestArray[k][1], ptestArray[k][2])
        innerpoints[k] = points
        z[threadid()] = []
    end

    return innerpoints
end

internalpoints

In [40]:
"""
    bool3d(assembly)

# Arguments
- `assembly::LinearAlgebraicRepresentation.Struct`

# Returns
- `W::Matrix{Float64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`
- `copCF::SparseMatrixCSC{Int8, Int64}`
- `boolmatrix::BitMatrix`

# Usage

```julia
using LARgenerators, SparseArrays, Base.Threads
import LinearAlgebraicRepresentation as Lar

n, m, p = 1, 1, 1
V, (VV, EV, FV, CV) = Lar.cuboidGrid([n, m, p], true)
cube = V, FV, EV

assembly = Lar.Struct([ cube,
    Lar.t(.3,.4,.25), Lar.r(pi/5,0,0), Lar.r(0,0,pi/12), cube,
    Lar.t(-.2,.4,-.2), Lar.r(0,pi/5,0), Lar.r(0,pi/12,0), cube
])

W, copEV, copFE, copCF, boolmatrix = LARgenerators.bool3d(assembly)
```

# Explanation 
1. After the arrangement, extract all the d-cells from (d-1)-coboundary as isolated polyhedra.
2. Then compute a single interior point for each of them.
3. Then compare each such point against all input boundaries, in order to compute those which it was interior to. Extend this point membership as 3-cell containment within the relative input solids.
4. The point membership with a boundary consists in the parity count of the intersection points of a vertical ray starting at the test point, with the boundary surface.
"""
function bool3d(assembly)

    V, FV, EV = Lar.struct2lar(assembly)
    cop_EV = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
    cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells)
    W = convert(Lar.Points, V')

    Z::Matrix{Float64}, copEV::SparseMatrixCSC{Int8,Int64}, copFE::SparseMatrixCSC{Int8,Int64}, copCF::SparseMatrixCSC{Int8,Int64} = Lar.space_arrangement(W, cop_EV, cop_FE)
    W = convert(Lar.Points, Z')

    innerpoints = internalpoints(W, copEV, copFE, copCF[2:end, :])

    listOfModels = Lar.evalStruct(assembly)
    inputfacenumbers = [length(listOfModels[k][2]) for k = 1:length(listOfModels)]
    cumulative = cumsum([0; inputfacenumbers]) .+ 1
    fspans = collect(zip(cumulative[1:end-1], cumulative[2:end] .- 1))
    span(h) = [j for j = 1:length(fspans) if fspans[j][1] <= h <= fspans[j][2]]

    containmenttest = testinternalpoint(V, EV, FV)
    boolmatrix = BitArray(undef, length(innerpoints) + 1, length(fspans) + 1)
    boolmatrix[1, 1] = 1

    for (k, point) in collect(enumerate(innerpoints))
        cells = containmenttest(point)
        rows = [span(h) for h in cells]
        for l in GL.Cat(rows)
            boolmatrix[k+1, l+1] = 1
        end
    end
    return W, copEV, copFE, copCF, boolmatrix
end

bool3d

## Esempio versione originale

In [41]:
using LARgenerators, SparseArrays, Base.Threads
import ViewerGL as GL
import LinearAlgebraicRepresentation as Lar

T = 3

n, m, p = 1, 1, 1
V, (VV, EV, FV, CV) = Lar.cuboidGrid([n, m, p], true)
cube = V, FV, EV

#= assembly = Lar.Struct([
    Lar.Struct([Lar.t(rand(-0.4:0.1:0.4), rand(-0.4:0.1:0.4), rand(-0.4:0.1:0.4)), Lar.r(rand(-0.5:0.1:0.5), 0, 0), cube]) for _ in 1:T
]) =#

assembly = Lar.Struct([ cube,
    Lar.t(.3,.4,.25), Lar.r(pi/5,0,0), Lar.r(0,0,pi/12), cube,
    Lar.t(-.2,.4,-.2), Lar.r(0,pi/5,0), Lar.r(0,pi/12,0), cube
])


V, FV = Lar.struct2lar(assembly)

meshes = []
for k = 1:length(FV)
    color = GL.MayaColors[k%12+1] - (rand(Float64, 4) * 0.1)
    push!(meshes, GL.GLGrid(V, [FV[k]], color, 0.9))
end
GL.VIEW(meshes);

W, copEV, copFE, copCF, boolmatrix = LARgenerators.bool3d(assembly)
Matrix(boolmatrix)

A = boolmatrix[:, 2]
B = boolmatrix[:, 3]
C = boolmatrix[:, 4]

AorBorC = A .| B .| C
AandBandC = A .& B .& C
AxorBxorC = A .⊻ B .⊻ C
AminBminC = .&(A, .!B, .!C)

difference = Matrix(copCF)' * Int.(AandBandC)
xor = Matrix(copCF)' * Int.(AxorBxorC)
or = Matrix(copCF)' * Int.(AorBorC)
min = Matrix(copCF)' * Int.(AminBminC)

V, CVs, FVs, EVs = Lar.pols2tria(W, copEV, copFE, copCF, difference) # part of assembly

GL.VIEW(GL.GLExplode(V, FVs, 1.01, 1.01, 1.01, 99, 0.5));
GL.VIEW(GL.GLExplode(V, EVs, 1, 1, 1, 1, 1));

EVs = [[[13, 14], [16, 17], [13, 17], [14, 16]], [[22, 26], [13, 14], [22, 23], [14, 23], [13, 26]], [[22, 23], [23, 28], [22, 28]], [[13, 17], [17, 40], [13, 26], [26, 40]], [[22, 26], [28, 43], [40, 43], [26, 40], [22, 28]], [[16, 17], [17, 40], [16, 43], [40, 43]], [[28, 43], [14, 16], [14, 23], [23, 28], [16, 43]]]
triangulated_faces = Any[[[14, 13, 16], [17, 16, 13]], [[26, 22, 13], [23, 14, 13], [23, 13, 22]], [[22, 28, 23]], [[17, 13, 40], [26, 40, 13]], [[26, 22, 40], [40, 28, 43], [28, 40, 22]], [[17, 16, 40], [43, 40, 16]], [[14, 16, 23], [23, 43, 28], [43, 23, 16]]]


## Performance del codice sviluppato ed ottimizzato

In [24]:
using BenchmarkTools
@btime bool3d($assembly)

  68.442 ms (445714 allocations: 24.09 MiB)


([0.0 0.0 … 0.3523450000000002 1.103010336951418; 0.0 0.0 … 2.1582775 1.951098135777665; 0.0 1.0 … 0.31964459999999983 0.9470026676010307], sparse([1, 5, 23, 1, 6, 24, 2, 5, 29, 4  …  83, 78, 81, 86, 79, 80, 87, 79, 81, 88], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  46, 47, 47, 47, 48, 48, 48, 49, 49, 49], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, -1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 88, 49), sparse([1, 7, 1, 9, 2, 10, 1, 13, 1, 15  …  40, 45, 39, 46, 41, 47, 42, 44, 42, 47], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  84, 84, 85, 85, 86, 86, 87, 87, 88, 88], Int8[1, 1, -1, 1, 1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, -1, -1, 1, 1, 1], 47, 88), sparse([1, 5, 2, 7, 1, 5, 3, 6, 4, 8  …  5, 7, 1, 2, 3, 4, 6, 8, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  43, 43, 44, 44, 45, 45, 46, 46, 47, 47], Int8[-1, 1, -1, 1, 1, -1, -1, 1, -1, 1  …  -1, 1, -1, 1, 1, -1, 1, -1, 1, -1], 8, 47), Bool[1 0 0 0; 0 0 0 1; … ; 0 1 0 1; 0 0 1 1])

## Performance del codice originale

In [25]:
@btime bool3dEx($assembly)

  70.804 ms (455029 allocations: 24.52 MiB)


([0.0 0.0 … 0.3523450000000002 1.103010336951418; 0.0 0.0 … 2.1582775 1.951098135777665; 0.0 1.0 … 0.31964459999999983 0.9470026676010307], (sparse([1, 5, 23, 1, 6, 24, 2, 5, 29, 4  …  83, 78, 81, 86, 79, 80, 87, 79, 81, 88], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  46, 47, 47, 47, 48, 48, 48, 49, 49, 49], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, -1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 88, 49), sparse([1, 7, 1, 9, 2, 10, 1, 13, 1, 15  …  40, 45, 39, 46, 41, 47, 42, 44, 42, 47], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  84, 84, 85, 85, 86, 86, 87, 87, 88, 88], Int8[1, 1, -1, 1, 1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, -1, -1, 1, 1, 1], 47, 88), sparse([1, 5, 2, 7, 1, 5, 3, 6, 4, 8  …  5, 7, 1, 2, 3, 4, 6, 8, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  43, 43, 44, 44, 45, 45, 46, 46, 47, 47], Int8[-1, 1, -1, 1, 1, -1, -1, 1, -1, 1  …  -1, 1, -1, 1, 1, -1, 1, -1, 1, -1], 8, 47)), Bool[1 0 0 0; 0 0 0 1; … ; 0 1 0 1; 0 0 1 1])

## Osservazioni sulle perfomance
Le prestazioni di `bool3d()` sono strettamente influenzate dalla funzione `space_arrangement()`

In [26]:
V, FV, EV = Lar.struct2lar(assembly)
cop_EV = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells)
W = convert(Lar.Points, V')

@btime Lar.space_arrangement($W, $cop_EV, $cop_FE)

  54.936 ms (330254 allocations: 18.54 MiB)


([0.0 0.0 0.0; 0.0 0.0 1.0; … ; 0.3523450000000002 2.1582775 0.31964459999999983; 1.103010336951418 1.951098135777665 0.9470026676010307], sparse([1, 5, 23, 1, 6, 24, 2, 5, 29, 4  …  83, 78, 81, 86, 79, 80, 87, 79, 81, 88], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  46, 47, 47, 47, 48, 48, 48, 49, 49, 49], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, -1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 88, 49), sparse([1, 7, 1, 9, 2, 10, 1, 13, 1, 15  …  40, 45, 39, 46, 41, 47, 42, 44, 42, 47], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  84, 84, 85, 85, 86, 86, 87, 87, 88, 88], Int8[1, 1, -1, 1, 1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, -1, -1, 1, 1, 1], 47, 88), sparse([1, 5, 2, 7, 1, 5, 3, 6, 4, 8  …  5, 7, 1, 2, 3, 4, 6, 8, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  43, 43, 44, 44, 45, 45, 46, 46, 47, 47], Int8[-1, 1, -1, 1, 1, -1, -1, 1, -1, 1  …  -1, 1, -1, 1, 1, -1, 1, -1, 1, -1], 8, 47))